## Hands-on Tutorial: Quantifying and Reducing Gender Stereotypes in Word Embeddings

Ensuring fairness in algorithmically-driven decision-making is important to avoid inadvertent cases of bias and perpetuation of harmful stereotypes. However, modern natural language processing techniques, which learn model parameters based on data, might rely on implicit biases presented in the data to make undesirable stereotypical associations. Such a danger is facing us with word embedding, a popular framework to represent text data as vectors which has been used in many machine learning and natural language processing tasks. Recent results ([1](https://arxiv.org/abs/1607.06520), [2](https://arxiv.org/abs/1608.07187)) show that even word embeddings trained on Google News articles exhibit female/male gender stereotypes to a disturbing extent. This raises concerns because of their widespread use, as we describe, often tends to amplify these biases. 

In the following, we provide step-by-step instructions to demonstrate and quanitfy the biases in word embedding.



In [9]:
# Setup:
# Clone the code repository from https://github.com/tolga-b/debiaswe.git
# mkdir debiaswe_tutorial
# cd debiaswe_tutorial
# git clone https://github.com/tolga-b/debiaswe.git

# To reduce the time of downloading data, we provide as subset of GoogleNews-vectors in the following location:
# https://drive.google.com/file/d/1NH6jcrg8SXbnhpIXRIXF_-KUE7wGxGaG/view?usp=sharing

# For full embeddings:
# Download embeddings at https://github.com/tolga-b/debiaswe and put them on the following directory
# embeddings/GoogleNews-vectors-negative300-hard-debiased.bin
# embeddings/GoogleNews-vectors-negative300.bin

In [1]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions
import re
import tqdm

### Downlaod the itwac embeddings from [here](https://drive.google.com/file/d/1xSxLkf7KgFVPaR3mlKxgWORosAwyIcs8/view?usp=sharing) or execute the code below

## Part 1: Gender Bias in Word Embedding


### Step 1: Load data
We first load the word embedding trained on a corpus of Google News texts consisting of 3 million English words and terms. The embedding maps each word into a 300-dimension vector. 

In [4]:
# words = {}

# for i in E.words:
#     if re.match(r"\b[a-zA-Z]+\b", i):
#         words[i.lower()] = E.v(i)

# with open('./embeddings/filtered_itwac128.tsv',"w") as file:
#     for word, vec in tqdm.tqdm(words.items()):
#         string_to_write = "\t".join([word] + list(map(str, list(vec))))+"\n"
#         file.write(string_to_write, )


In [65]:
E = WordEmbedding('./embeddings/filtered_itwac128.tsv')

*** Reading data from ./embeddings/filtered_itwac128.tsv
(916686, 128)
916686 words of dimension 128 : di, e, che, la, ..., minore-genitore/i, babelzilla, spaziolaboratorio, eorismo


### Step 2: Define gender direction

We define gender direction by the direciton of she - he because they are frequent and do not have fewer alternative word senses (e.g., man can also refer to mankind). In the paper, we discuss alternative approach for defining gender direction (e.g., using PCA).

In [66]:
# compute gender direction
female_words = ["lei", "donna", "madre", "moglie", "sorella", "femmina"]

male_words = ["lui", "uomo", "padre", "marito", "fratello", "maschio"]
definitional = list(zip(male_words, female_words))

gender_direction = we.doPCA(definitional, E).components_[0]

### Step 3: Generating analogies of "Man: x :: Woman : y"

We show that the word embedding model generates gender-streotypical analogy pairs. 
To generate the analogy pairs, we use the analogy score defined in our paper. This score finds word pairs that are well aligned with gender direction as well as within a short distance from each other to preserve topic consistency. 


In [67]:
# analogies gender
a_gender = E.best_analogies_dist_thresh(gender_direction, max_words = 50000)

Computing neighbors
Mean: 183.51192
Median: 71.0


In [57]:
for (a,b,c) in a_gender:
    # if c > 0.6:
    print(a+"-"+b+" score: "+str(c))


lettore-lettrice score: 1.0
operaio-operaia score: 1.0
attore-attrice score: 1.0
medico-medica score: 1.0
direttore-direttrice score: 1.0
ottico-ottica score: 1.0
idraulico-idraulica score: 1.0
psicologo-psicologa score: 1.0
politico-politica score: 1.0
pittore-pittrice score: 1.0
bancario-bancaria score: 1.0
coreografo-coreografa score: 1.0
segretario-segretaria score: 1.0
bibliotecario-bibliotecaria score: 1.0
operatore-operatrice score: 1.0
addetto-addetta score: 1.0
veterinario-veterinaria score: 1.0
artigiano-artigiana score: 1.0
meccanico-meccanica score: 1.0
impiegato-impiegata score: 1.0
storico-storica score: 1.0
formatore-pedagogista score: 0.72651446
orafo-falegnameria score: 0.65940297
sacerdote-maestra score: 0.6481215
sarto-porcellana score: 0.6402431
scultore-autoritratto score: 0.6368869
calciatore-doppietta score: 0.63425905
padre-madre score: 0.6336403
ballerino-gemello score: 0.6303773
geologo-geometra score: 0.6234609
uomo-donna score: 0.62036294
biologo-psicoterape

### Step 4: Analyzing gender bias in word vectors asscoiated with professions

Next, we show that many occupations are unintendedly associated with either male of female by projecting their word vectors onto the gender dimension. 

The script will output the profession words sorted with respect to the projection score in the direction of gender.

In [68]:
faculties = ['infermieristica',
             'farmacia',
             'medicina',
             'chirurgia',
             'odontoiatria',
             'biotecnologie',
             'ingegneria',
             'biologia',
             'architettura',
             'agraria',
             'chimica',
             'geologia',
             'statistica',
             'design',
             'fisica',
             'informatica',
             'biomedica',
             'automazione',
             'elettronica',
             'gestionale',
             'matematica',
             'geofisica',
             'zootecnica',
             'comunicazione',
             'psicologia',
             'economia',
             'sociologia',
             'criminologia',
             'giurisprudenza',
             'finanza',
             'pedagogia',
             'filosofia',
             'geografia',
             'lettere',
             'lingue',
             'storia',
             'antropologia',
             'archeologia',
             'filologia',
             'letteratura',
             'linguistica',
             'musicologia',
             'arte']


In [70]:
# profession analysis gender
sp = sorted([(E.v(w).dot(gender_direction), w) for w in faculties])
sp

[(-0.043069378, 'ingegneria'),
 (-0.0160173, 'zootecnica'),
 (-0.0160173, 'zootecnica'),
 (0.024629425, 'design'),
 (0.02660481, 'infermieristica'),
 (0.038775083, 'sociologia'),
 (0.040885232, 'gestionale'),
 (0.05979086, 'automazione'),
 (0.06100458, 'statistica'),
 (0.06147439, 'economia'),
 (0.06201877, 'chirurgia'),
 (0.066020474, 'geologia'),
 (0.06653127, 'criminologia'),
 (0.07221107, 'matematica'),
 (0.07242691, 'geofisica'),
 (0.07685487, 'filologia'),
 (0.077737585, 'elettronica'),
 (0.07874751, 'odontoiatria'),
 (0.09490307, 'informatica'),
 (0.09776217, 'architettura'),
 (0.10769771, 'pedagogia'),
 (0.11240166, 'biomedica'),
 (0.1137523, 'finanza'),
 (0.11678499, 'fisica'),
 (0.12125731, 'psicologia'),
 (0.12391965, 'antropologia'),
 (0.13961963, 'giurisprudenza'),
 (0.1411029, 'linguistica'),
 (0.14112332, 'filosofia'),
 (0.15114108, 'biotecnologie'),
 (0.15182716, 'biologia'),
 (0.15623777, 'chimica'),
 (0.15735163, 'arte'),
 (0.15793613, 'agraria'),
 (0.16332456, 'music

## Exercise

Repeat Step 2-4 with debiased word embedding. 

You can use debiaswe debias function to do the debiasing with word sets of your choosing

You can leave equalize_pairs and gender_specific_words blank when coming up with your own groups. We give an example for the case of gender below for you to warm up.

In [60]:
from debiaswe.debias import debias

In [61]:
# Lets load some gender related word lists to help us with debiasing
defs = definitional

with open('./data/equalize_pairs_it.json', "r") as f:
    equalize_pairs = json.load(f)

with open('./data/gender_specific_seed.json', "r") as f:
    gender_specific_words = json.load(f)
print("gender specific", len(gender_specific_words), gender_specific_words[:10])

gender specific 218 ['actress', 'actresses', 'aunt', 'aunts', 'bachelor', 'ballerina', 'barbershop', 'baritone', 'beard', 'beards']


In [62]:
def debias(E, biased_words, definitional, equalize):
    gender_direction = we.doPCA(definitional, E).components_[0]
    
    for i, w in enumerate(E.words):
        # For each word which is not gender specific, we remove the gendere direction.
        if w in biased_words:
            E.vecs[i] = we.drop(E.vecs[i], gender_direction)
    E.normalize()
    candidates = {x for e1, e2 in equalize for x in [(e1.lower(), e2.lower()),
                                                     (e1.title(), e2.title()),
                                                     (e1.upper(), e2.upper())]}
    # print(candidates)
    for (a, b) in candidates:
        #  Centrano il centro della coppia delle parole gender specific.
        
        if (a in E.index and b in E.index):
            y = we.drop((E.v(a) + E.v(b)) / 2, gender_direction)
            z = np.sqrt(1 - np.linalg.norm(y)**2)
            if (E.v(a) - E.v(b)).dot(gender_direction) < 0:
                z = -z
            E.vecs[E.index[a]] = z * gender_direction + y
            E.vecs[E.index[b]] = -z * gender_direction + y
    E.normalize()

In [63]:
debias(E, faculties, defs, equalize_pairs)

916686 words of dimension 128 : di, e, che, la, ..., minore-genitore/i, babelzilla, spaziolaboratorio, eorismo
916686 words of dimension 128 : di, e, che, la, ..., minore-genitore/i, babelzilla, spaziolaboratorio, eorismo


In [64]:
sp_debiased = sorted([(E.v(w).dot(gender_direction), w) for w in faculties])
sp_debiased

[(-1.4901161e-08, 'biologia'),
 (-7.450581e-09, 'antropologia'),
 (-7.450581e-09, 'geofisica'),
 (-7.450581e-09, 'linguistica'),
 (-5.5879354e-09, 'design'),
 (-3.7252903e-09, 'criminologia'),
 (-3.7252903e-09, 'ingegneria'),
 (-3.7252903e-09, 'matematica'),
 (-3.7252903e-09, 'musicologia'),
 (-3.7252903e-09, 'pedagogia'),
 (-3.7252903e-09, 'sociologia'),
 (-1.8626451e-09, 'giurisprudenza'),
 (-1.8626451e-09, 'informatica'),
 (-1.8626451e-09, 'odontoiatria'),
 (0.0, 'agraria'),
 (0.0, 'archeologia'),
 (0.0, 'arte'),
 (0.0, 'biotecnologie'),
 (0.0, 'fisica'),
 (0.0, 'geografia'),
 (0.0, 'geologia'),
 (0.0, 'medicina'),
 (0.0, 'storia'),
 (1.8626451e-09, 'chirurgia'),
 (3.7252903e-09, 'automazione'),
 (3.7252903e-09, 'economia'),
 (3.7252903e-09, 'filologia'),
 (3.7252903e-09, 'infermieristica'),
 (3.7252903e-09, 'letteratura'),
 (3.7252903e-09, 'statistica'),
 (5.5879354e-09, 'architettura'),
 (7.450581e-09, 'biomedica'),
 (7.450581e-09, 'chimica'),
 (7.450581e-09, 'elettronica'),
 (7.4

In [18]:
# profession analysis gender
sp_debiased = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp_debiased[0:20], sp_debiased[-20:]

([(-0.4196325, 'congressman'),
  (-0.40675846, 'businessman'),
  (-0.32398778, 'councilman'),
  (-0.30967093, 'dad'),
  (-0.21665451, 'statesman'),
  (-0.11345412, 'salesman'),
  (-0.07300486, 'monk'),
  (-0.072163954, 'handyman'),
  (-0.04946825, 'minister'),
  (-0.043583885, 'archbishop'),
  (-0.040207203, 'bishop'),
  (-0.038332462, 'commissioner'),
  (-0.035724398, 'surgeon'),
  (-0.03313398, 'trader'),
  (-0.032377202, 'observer'),
  (-0.032095842, 'neurosurgeon'),
  (-0.03145011, 'priest'),
  (-0.031133963, 'skipper'),
  (-0.029659165, 'lawmaker'),
  (-0.029511197, 'commander')],
 [(0.029965654, 'teenager'),
  (0.030237045, 'instructor'),
  (0.030946147, 'student'),
  (0.03111697, 'paralegal'),
  (0.032039426, 'bookkeeper'),
  (0.03243462, 'cinematographer'),
  (0.034329068, 'graphic_designer'),
  (0.03470566, 'lifeguard'),
  (0.035666533, 'janitor'),
  (0.035971954, 'drummer'),
  (0.04212016, 'wrestler'),
  (0.04390225, 'hairdresser'),
  (0.04813315, 'firefighter'),
  (0.2377612

In [11]:
# analogies gender
a_gender_debiased = E.best_analogies_dist_thresh(v_gender)

for (a,b,c) in a_gender_debiased:
    print(a+"-"+b)

Computing neighbors
Mean: 10.218597434053665
Median: 7.0
females-males
ovarian_cancer-prostate_cancer
estrogen-testosterone
grandmother-grandfather
girl-boy
moms-dads
councilwoman-councilman
daughter-son
filly-colt
chairwoman-chairman
gals-dudes
she-he
women-men
girls-boys
granddaughters-grandsons
mare-gelding
queen-king
mothers-fathers
princess-prince
convent-monastery
sorority-fraternity
grandma-grandpa
husbands-wives
twin_sister-twin_brother
sister-brother
sisters-brothers
herself-himself
her-his
female-male
congresswoman-congressman
schoolgirl-schoolboy
businesswoman-businessman
motherhood-fatherhood
ex_boyfriend-ex_girlfriend
aunt-uncle
mother-father
niece-nephew
mom-dad
granddaughter-grandson
woman-man
spokeswoman-spokesman
queens-kings
ladies-gentlemen
daughters-sons
actress-actor
lesbian-gay
compatriot-countryman
husband-younger_brother
gal-dude
hers-theirs
heroine-protagonist
feminism-feminist
actresses-actors
childhood-boyhood
waitress-waiter
kid-guy
me-him
mommy-daddy
aunts-